# SQL speed drilling + hypothesis testing intro

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [2]:
df = pd.read_csv('../../data/raw/listings.csv')
pd.set_option('display.max_rows', None)
print(f"Dataset shape: {df.shape}")
# print(f"Columns available: {df.columns.tolist()}")
print(f"Price dtype: {df['price'].dtype}")
print(f"Neighbourhood dtype: {df['neighbourhood_cleansed'].dtype}")
df.head(1).T 

Dataset shape: (15187, 79)
Price dtype: object
Neighbourhood dtype: int64


,0
id,5456
listing_url,https://www.airbnb.com/rooms/5456
scrape_id,20250613040113
last_scraped,2025-06-13
source,city scrape
name,"Walk to 6th, Rainey St and Convention Ctr"
description,Great central location for walking to Convent...
neighborhood_overview,My neighborhood is ideally located if you want...
picture_url,https://a0.muscache.com/pictures/14084884/b5a3...
host_id,8028


In [3]:
df['price'] = df['price'].str.replace(r'[$,]', '', regex=True).astype(float)

print(f"Price dtype after cleaning: {df['price'].dtype}")
print(f"Price range: ${df['price'].min():.2f} to ${df['price'].max():.2f}")

print(f"✅ Price cleaned: {df['price'].dtype}")
print(f"✅ Sample prices: {df['price'].head().tolist()}")

df['price'].describe()

Price dtype after cleaning: float64
Price range: $9.00 to $50000.00
✅ Price cleaned: float64
✅ Sample prices: [101.0, 45.0, nan, 155.0, 43.0]


count    10708.000000
mean       386.470583
std       2620.198322
min          9.000000
25%         89.000000
50%        138.000000
75%        235.000000
max      50000.000000
Name: price, dtype: float64

# SQL Drills

In [4]:
import sqlite3 as sq

conn = sq.connect(':memory:')
df.to_sql('austin_housing', conn, index=False)

15187

## Query 1: Top Revenue Neighborhoods
**Business question: Which 5 neighborhoods generate the highest average estimated revenue? Show neighborhood, average revenue, and count of listings.**

Requirements:
- Group by neighborhood
- Calculate mean revenue and count
- Sort by average revenue (highest first)
- Show only top 5

In [5]:
t1q = """ SELECT host_neighbourhood,
    AVG(estimated_revenue_l365d) avg_annual_revenue,
    COUNT(listing_url) as listings_count
    FROM austin_housing
    GROUP BY host_neighbourhood
    ORDER BY avg_annual_revenue DESC
"""

t1r = pd.read_sql(t1q, conn)
t1r.head(5)

,host_neighbourhood,avg_annual_revenue,listings_count
0,Gateway District,369230.769231,90
1,Historic District,182865.000000,2
2,Touro,162690.000000,1
3,Eck Lane On The Lake,162576.000000,1
4,Rolling Hills West,139740.000000,1


## Query 2: Superhost Premium

**Business question: Do superhosts earn more? Compare average nightly price between superhosts and non-superhosts.**

### Show the count of each group and the price difference.
**Requirements:**
- Group by host_is_superhost
- Calculate average price and count for each group
- Show both groups

In [6]:
t2q = """ SELECT host_is_superhost,
    COUNT(*),
    AVG(price) as avg_price
    FROM austin_housing
    GROUP BY host_is_superhost
"""

t2r = pd.read_sql(t2q, conn)
t2r.head()

,host_is_superhost,COUNT(*),avg_price
0,None,478,183.640587
1,f,8856,552.065084
2,t,5853,242.663806


## Query 3: High-Revenue Full Properties
**Business question: Find entire home/apt listings that generate above-average revenue AND have high review scores (4.8+). Show property type, price, revenue, and review score.**

### Requirements:
- Filter: room_type = 'Entire home/apt'
- Filter: estimated_revenue_l365d > overall average
- Filter: review_scores_rating >= 4.8
- Show: property_type, price, estimated_revenue_l365d, review_scores_rating
- Sort by revenue descending
- Limit to 10 results

**This one combines WHERE with multiple conditions**

In [7]:
t3q = """ SELECT property_type,
    price,
    review_scores_rating,
    estimated_revenue_l365d
    FROM austin_housing
    WHERE (review_scores_rating >= 4.8) & (room_type == 'Entire home/apt')
    GROUP BY property_type, price, review_scores_rating, estimated_revenue_l365d
    HAVING estimated_revenue_l365d >= AVG(estimated_revenue_l365d)
    ORDER BY estimated_revenue_l365d DESC
"""

t3r = pd.read_sql(t3q, conn)

t3r.head(10)

,property_type,price,review_scores_rating,estimated_revenue_l365d
0,Entire home,2250.0,4.93,499500.0
1,Entire home,1350.0,5.00,307800.0
2,Entire serviced apartment,50000.0,5.00,300000.0
3,Entire home,1699.0,5.00,285432.0
4,Entire home,1046.0,4.90,266730.0
5,Entire home,895.0,4.92,228225.0
6,Entire home,1670.0,4.89,220440.0
7,Entire home,915.0,4.94,214110.0
8,Entire home,814.0,4.93,207570.0
9,Entire home,913.0,4.95,202686.0


## Query 4: Neighborhood Capacity Analysis

**Business question: Which neighborhoods have the most available properties (high availability_365) with at least 3 bedrooms? Show neighborhood, average availability, count of properties, and average price.**

### Requirements:
- Filter: bedrooms >= 3
- Group by neighbourhood_cleansed
- Show: neighborhood, AVG(availability_365), COUNT(*), AVG(price)
- Sort by average availability descending
- Top 10

**This tests GROUP BY + aggregations.**

In [8]:
t4q = """ SELECT neighbourhood_cleansed,
    AVG(price) as avg_price,
    AVG(availability_365) as avg_availability,
    COUNT(*)
    FROM austin_housing
    WHERE bedrooms >= 3
    GROUP BY neighbourhood_cleansed
    ORDER BY avg_availability DESC
"""

t4r = pd.read_sql(t4q, conn)
t4r.head(10)

,neighbourhood_cleansed,avg_price,avg_availability,COUNT(*)
0,78742,299.000000,292.000000,1
1,78730,1317.200000,256.409091,22
2,78737,681.862069,254.376238,101
3,78733,1169.119048,254.000000,48
4,78732,1245.375000,241.795918,49
5,78734,513.479769,239.995122,205
6,78701,488.352941,233.927273,55
7,78738,3497.640000,230.000000,30
8,78736,443.050000,224.020833,48
9,78753,228.517857,220.358209,67


## Query 5: Minimum Stay Strategy Analysis

**Business question: Find neighborhoods where properties require longer minimum stays (avg minimum_nights > 7) AND generate above $20K annual revenue. Show neighborhood, average minimum nights, count of properties, and average revenue.**

### Requirements:
- Group by neighbourhood_cleansed
- Calculate AVG(minimum_nights), COUNT(*), AVG(estimated_revenue_l365d)
- Filter grouped results: average minimum_nights > 7
- Filter grouped results: average revenue > 20000
- Sort by average revenue descending
- Top 10

**This tests HAVING clause (filtering AFTER grouping)**

In [9]:
t5q = """ SELECT neighbourhood_cleansed,
    AVG(minimum_nights) as minimum_night_avg,
    COUNT(*) as listing_count,
    AVG(estimated_revenue_l365d) as avg_revenue
    FROM austin_housing
    GROUP BY neighbourhood_cleansed
    HAVING (minimum_night_avg > 7) AND (avg_revenue > 20000)
    ORDER BY avg_revenue DESC
"""

t5r = pd.read_sql(t5q,conn)

t5r.head(10)

,neighbourhood_cleansed,minimum_night_avg,listing_count,avg_revenue
0,78730,15.783784,37,31611.375


## Final Query: Query 6 - Price vs Revenue Efficiency

**Business question: Find listings where estimated revenue per available night is highest (high efficiency). Show listing URL, price, availability_365, estimated_revenue, and calculate revenue_per_available_night.**

### Requirements:
- Calculate: estimated_revenue_l365d / availability_365 as revenue_per_available_night
- Filter: availability_365 > 0 (avoid division by zero)
- Sort by revenue_per_available_night descending
- Top 10

**This tests calculated columns in SELECT**

In [10]:
t6q = """ SELECT listing_url,
    price,
    availability_365,
    estimated_revenue_l365d,
    (estimated_revenue_l365d / availability_365) as revenue_per_available_night
    FROM austin_housing
    WHERE availability_365 > 0
    GROUP BY listing_url, price, availability_365, estimated_revenue_l365d
    ORDER BY revenue_per_available_night DESC
"""

t6r = pd.read_sql(t6q,conn)

t6r.head(10)

,listing_url,price,availability_365,estimated_revenue_l365d,revenue_per_available_night
0,https://www.airbnb.com/rooms/868630470918063876,344.0,1,53664.0,53664.000000
1,https://www.airbnb.com/rooms/813711378507210342,552.0,4,109296.0,27324.000000
2,https://www.airbnb.com/rooms/32898594,445.0,6,109470.0,18245.000000
3,https://www.airbnb.com/rooms/52130073,111.0,1,10656.0,10656.000000
4,https://www.airbnb.com/rooms/1092075558102479638,627.0,10,94050.0,9405.000000
5,https://www.airbnb.com/rooms/46513674,300.0,3,25200.0,8400.000000
6,https://www.airbnb.com/rooms/907383017689120037,848.0,20,152640.0,7632.000000
7,https://www.airbnb.com/rooms/974761930785930443,532.0,18,135660.0,7536.666667
8,https://www.airbnb.com/rooms/854345788191857434,50000.0,347,2400000.0,6916.426513
9,https://www.airbnb.com/rooms/1235810514262281190,379.0,7,47754.0,6822.000000


In [11]:
downtown = df[df['neighbourhood_cleansed'] == 78701]['price']
east_austin = df[df['neighbourhood_cleansed'] == 78741]['price']

print(f"Downtown: {len(downtown)} listings, mean = ${downtown.mean():.2f}")
print(f"East Austin: {len(east_austin)} listings, mean = ${east_austin.mean():.2f}")

Downtown: 1246 listings, mean = $1175.58
East Austin: 859 listings, mean = $179.98


In [12]:
from scipy import stats

# Remove NaN values (don't replace with 0)
downtown_clean = downtown.dropna()
east_austin_clean = east_austin.dropna()

# Verify cleaning
print(f"Downtown: {len(downtown_clean)} listings, mean = ${downtown_clean.mean():.2f}")
print(f"East Austin: {len(east_austin_clean)} listings, mean = ${east_austin_clean.mean():.2f}")

# Run t-test on cleaned data
t_stat, p_value = stats.ttest_ind(downtown_clean, east_austin_clean)
print(f"\nT-statistic: {t_stat:.2f}")
print(f"P-value: {p_value:.4f}")

Downtown: 831 listings, mean = $1175.58
East Austin: 553 listings, mean = $179.98

T-statistic: 3.60
P-value: 0.0003
